In [1]:
from estimate_underreporting import *
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('sample_data/sample_reports_df.csv', parse_dates=['SRCreatedDate', 'InsCreatedDate', 'WOCreatedDate']).head(10000)
df.head(5)

,SRID,IncidentGlobalID,SRCreatedDate,InsCreatedDate,WOCreatedDate,SRCategory,BoroughCode,RiskRating
0,23071289,E5416B67-0692-4888-AB29-AD0391DF2C53,2022-02-19 21:55:00,2022-02-25 14:54:17,NaT,Remove Tree,Queens,7.0
1,23071289,E5416B67-0692-4888-AB29-AD0391DF2C53,2022-02-19 21:55:00,2022-02-25 14:54:17,NaT,Remove Tree,Queens,8.0
2,23493187,F3C21D50-E891-4829-8243-A817DA218747,2022-05-02 05:30:39,2022-05-03 15:19:22,2022-05-03 15:20:12,Hazard,Bronx,9.0
3,24258242,1F58426B-0BB3-40DB-BFCE-8FC2341CB9EA,2022-08-15 17:35:43,2022-08-22 16:42:07,2022-08-22 16:44:17,Prune,Queens,8.0
4,24258242,1F58426B-0BB3-40DB-BFCE-8FC2341CB9EA,2022-08-15 17:35:43,2022-08-22 16:36:06,2022-08-22 16:38:14,Prune,Queens,NaN


In [6]:
incidents = create_incidents_df(df,
                        incident_identifier_col = 'IncidentGlobalID', 
                        observation_start_col = 'SRCreatedDate',
                        observation_end_col = ['InsCreatedDate', 'WOCreatedDate'], 
                        covariates_cont = 'RiskRating',
                        covariates_cat = ['SRCategory', 'BoroughCode'])
y, X, duration = prepare_data_for_regression(incidents, 
                                             covariate_cat = ['SRCategory', 'BoroughCode'],
                                             covariate_cont = ['RiskRating'],)
res_standard = train_model(y, X, duration, model_type='standard')

Optimization terminated successfully.
         Current function value: 0.545703
         Iterations: 128
         Function evaluations: 129
         Gradient evaluations: 129
                          Poisson Regression Results                          
Dep. Variable:         num_duplicates   No. Observations:                 4441
Model:                        Poisson   Df Residuals:                     4427
Method:                           MLE   Df Model:                           13
Date:                Thu, 12 Oct 2023   Pseudo R-squ.:                  0.1009
Time:                        17:49:08   Log-Likelihood:                -2423.5
converged:                       True   LL-Null:                       -2695.3
Covariance Type:            nonrobust   LLR p-value:                7.666e-108
                                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [7]:
res_zeroinflated = train_model(y, X, duration, model_type='zeroinflated')

Optimization terminated successfully.
         Current function value: 0.473045
         Iterations: 126
         Function evaluations: 127
         Gradient evaluations: 127
                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:          num_duplicates   No. Observations:                 4441
Model:             ZeroInflatedPoisson   Df Residuals:                     4427
Method:                            MLE   Df Model:                           13
Date:                 Thu, 12 Oct 2023   Pseudo R-squ.:                 0.08881
Time:                         17:49:12   Log-Likelihood:                -2100.8
converged:                        True   LL-Null:                       -2305.5
Covariance Type:             nonrobust   LLR p-value:                 2.188e-79
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------

In [8]:
incidents = append_delays_to_incidents_df(incidents,
                                          generate_predicted_reporting_delay(res_standard, X))

In [13]:
incidents.sample(10)

,IncidentGlobalID,num_duplicates,duration,RiskRating,SRCategory,BoroughCode,predicted_reporting_delay
3250,8D7225B1-98ED-4EBF-890E-E143B34B975D,0,2.764907,1.243670,Hazard,Queens,24.582881
3309,90623D5F-47E9-4940-9F9B-C4E967F6D99D,0,6.709410,-0.469817,Remove Tree,Queens,179.663057
1524,423B53FA-1792-4AD3-8FBF-A6E518A37F75,0,4.002685,-0.469817,Hazard,Bronx,61.527238
3097,8755487B-1B27-4BC7-9C2F-6662716AE5C7,0,3.866620,-0.469817,Illegal Tree Damage,Brooklyn,76.482014
538,17D8A531-0C46-4733-B496-05B81B966667,0,0.842361,2.385995,Hazard,Queens,16.599318
1517,4200BF08-A2CB-496E-9CE5-7C717C567105,0,0.975498,0.101346,Prune,Manhattan,91.682252
5377,E59B561D-156C-443F-9374-DEA9ACF0124A,0,2.922269,-0.469817,Illegal Tree Damage,Bronx,142.136827
2565,70C176A9-C014-451A-9785-D2D81198D2BF,0,0.661586,-2.183304,Hazard,Staten Island,75.399672
3912,A9095F09-A66D-4E38-8AEC-8914A6812EC4,0,0.555231,1.814833,Hazard,Brooklyn,15.095069
4612,C68608FE-8CD5-4003-A67F-67A38A2CF73D,0,0.940486,0.672508,Prune,Bronx,182.181132
